In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/DACON/credit/train.csv')
train = train.drop(['index'], axis=1)

test = pd.read_csv('/content/drive/MyDrive/DACON/credit/test.csv')
test = test.drop(['index'], axis=1)

submit = pd.read_csv('/content/drive/MyDrive/DACON/credit/sample_submission.csv')

In [ ]:
!pip install BorutaShap

     |████████████████████████████████| 358kB 16.3MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491630 sha256=bec831adb541fbd2ddf77bc96f1fcf0ccbeeb47c50b7464dc61a20117030867e
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [ ]:
#Binary variables
def binary(data):
  data['gender'] = data['gender'].replace(['F','M'],[0,1])
  print('gender :')
  print(data['gender'].value_counts())
  print('--------------')

  print('Having a car or not : ')
  data['car'] = data['car'].replace(['N','Y'],[0,1])
  print(data['car'].value_counts())
  print('--------------')

  print('Having house reality or not: ')
  data['reality'] = data['reality'].replace(['N','Y'],[0,1])
  print(data['reality'].value_counts())
  print('--------------')
        
  print('Having a phone or not: ')
  print(data['phone'].value_counts())
  print('--------------')
        

  print('Having a email or not: ')
  print(data['email'].value_counts())
  print('--------------')
        

  print('Having a work phone or not: ')
  print(data['work_phone'].value_counts())
  print('--------------')
  return

In [ ]:
binary(train)

gender :
0    17697
1     8760
Name: gender, dtype: int64
--------------
Having a car or not : 
0    16410
1    10047
Name: car, dtype: int64
--------------
Having house reality or not: 
1    17830
0     8627
Name: reality, dtype: int64
--------------
Having a phone or not: 
0    18672
1     7785
Name: phone, dtype: int64
--------------
Having a email or not: 
0    24042
1     2415
Name: email, dtype: int64
--------------
Having a work phone or not: 
0    20511
1     5946
Name: work_phone, dtype: int64
--------------


In [ ]:
from sklearn.preprocessing import LabelEncoder
def preprocess(data):
  label_encoder = LabelEncoder()
  data['income_type']=label_encoder.fit_transform(data['income_type'])
  data['edu_type']=label_encoder.fit_transform(data['edu_type'])
  data['family_type']=label_encoder.fit_transform(data['family_type'])
  data['house_type']=label_encoder.fit_transform(data['house_type'])
  return

In [ ]:
preprocess(train)

In [ ]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,0,0,0,202500.0,0,1,1,2,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,0,0,1,1,247500.0,0,4,0,1,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,1,1,1,0,450000.0,4,1,1,1,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,0,0,1,0,202500.0,0,4,1,1,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,0,1,1,0,157500.0,2,1,1,1,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,225000.0,2,4,1,1,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,0,0,1,1,180000.0,4,1,2,1,-15291,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,0,1,0,0,292500.0,4,4,0,5,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,1,0,1,0,171000.0,4,2,3,1,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


In [ ]:
X=train.drop('credit',axis=1)
X=X.drop('occyp_type',axis=1)
y=train['credit']

In [ ]:
y

0        1.0
1        1.0
2        2.0
3        0.0
4        2.0
        ... 
26452    1.0
26453    2.0
26454    2.0
26455    2.0
26456    2.0
Name: credit, Length: 26457, dtype: float64

In [ ]:
from BorutaShap import BorutaShap
from xgboost import XGBClassifier

model = XGBClassifier()
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=X, y=y, n_trials=100, sample=False,
            	     train_or_test = 'test', normalize=True,
		     verbose=True)

100%|██████████| 100/100 [15:02<00:00,  9.03s/it]

4 attributes confirmed important: ['DAYS_EMPLOYED', 'DAYS_BIRTH', 'income_total', 'begin_month']
11 attributes confirmed unimportant: ['child_num', 'FLAG_MOBIL', 'house_type', 'edu_type', 'gender', 'phone', 'email', 'family_type', 'family_size', 'car', 'work_phone']
2 tentative attributes remains: ['income_type', 'reality']


In [ ]:
self = Feature_Selector
if self.importance_measure == 'shap':

  self.explain()
  vals = self.shap_values

  vals = self.calculate_Zscore(vals)

  X_feature_import = vals[:len(self.X.columns)]
  Shadow_feature_import = vals[len(self.X_shadow.columns):]


elif self.importance_measure == 'gini':

  feature_importances_ =  np.abs(self.model.feature_importances_)

  feature_importances_ = self.calculate_Zscore(feature_importances_)

  X_feature_import = feature_importances_[:len(self.X.columns)]
  Shadow_feature_import = feature_importances_[len(self.X.columns):]

else:

  raise ValueError('No Importance_measure was specified select one of (shap, gini)')